In [41]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

In [42]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud

In [43]:
df = pd.read_excel(r'C:\Users\Praveen Shahani\Downloads\Copy of Data Analyst - Test Data.xlsx')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Review      6393 non-null   object        
 1   date        6448 non-null   datetime64[ns]
 2   Location    1711 non-null   object        
 3   Unnamed: 3  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 201.6+ KB


In [45]:
df.shape

(6448, 4)

In [46]:
df['Review'] = df['Review'].str.replace('[^\w\s]', '')
df['Review'] = df['Review'].str.replace('\d', '')

C:\Users\Praveen Shahani\AppData\Local\Temp\ipykernel_16280\1167260919.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Review'] = df['Review'].str.replace('[^\w\s]', '')
C:\Users\Praveen Shahani\AppData\Local\Temp\ipykernel_16280\1167260919.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Review'] = df['Review'].str.replace('\d', '')


In [47]:
sw = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

In [48]:
drops = pd.Series(' '.join(df['Review']).split()).value_counts()[-250:]
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in drops))

In [49]:
df["Review"].apply(lambda x: TextBlob(x).words).head()

0    [I, impressed, resort, Great, staff, main, res...
1    [The, rooms, nice, outside, needs, work, also,...
2    [Great, location, I, stayed, hotel, last, thre...
3    [The, hotel, adequate, stay, The, strips, bath...
4    [Great, location, room, large, spacious, Parki...
Name: Review, dtype: object

In [50]:
import nltk
nltk.download('wordnet')  

[nltk_data] Downloading package wordnet to C:\Users\Praveen
[nltk_data]     Shahani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
df['Review'] = df['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [52]:
df["Review"].head()

0    I impressed resort Great staff main resort poo...
1    The room nice outside need work also free brea...
2    Great location I stayed hotel last three trip ...
3    The hotel adequate stay The strip bathtub usel...
4    Great location room large spacious Parking eas...
Name: Review, dtype: object

In [53]:
tf = df["Review"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()

In [54]:
tf.columns = ["words", "tf"]
tf.head()

,words,tf
0,staff,1479.0
1,resort,399.0
2,Read,10612.0
3,reasonable,44.0
4,bar,186.0


In [55]:
tf.shape

(14711, 2)

In [56]:
tf["words"].nunique()

14711

In [57]:
tf["tf"].describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

count    14711.000000
mean        11.351914
std        121.970192
min          1.000000
5%           1.000000
10%          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
80%          5.000000
90%         13.000000
95%         33.000000
99%        164.000000
max      10612.000000
Name: tf, dtype: float64

In [58]:
tf.sort_values("tf", ascending = False).head(20)

,words,tf
2,Read,10612.0
9,le,5409.0
28,room,4099.0
11,I,3527.0
21,The,3302.0
29,hotel,2731.0
0,staff,1479.0
41,stay,1301.0
128,clean,1222.0
7,location,1199.0


In [59]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("I liked the food")

{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.4215}

In [60]:
df["Review"].apply(lambda x: x.upper())

0       I IMPRESSED RESORT GREAT STAFF MAIN RESORT POO...
1       THE ROOM NICE OUTSIDE NEED WORK ALSO FREE BREA...
2       GREAT LOCATION I STAYED HOTEL LAST THREE TRIP ...
3       THE HOTEL ADEQUATE STAY THE STRIP BATHTUB USEL...
4       GREAT LOCATION ROOM LARGE SPACIOUS PARKING EAS...
                              ...                        
6443    STAFF FRIENDLY FRANK CHEF BUFFET BAR TOP MEETI...
6444                    INTERNET BARELY WORKED DIDNT WORK
6445            LOCATION GREAT BED UNCOMFORTABLE BED EVER
6446    EARLY CHECK STAFF ACCOMMODATING ABLE CHECK SEV...
6447    STAFF FREE COFFEE LOCATION ACTIVITY BED SMALLE...
Name: Review, Length: 6448, dtype: object

In [61]:
df["Review"][0:10].apply(lambda x: sia.polarity_scores(x))

0    {'neg': 0.0, 'neu': 0.517, 'pos': 0.483, 'comp...
1    {'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'comp...
2    {'neg': 0.0, 'neu': 0.728, 'pos': 0.272, 'comp...
3    {'neg': 0.112, 'neu': 0.743, 'pos': 0.145, 'co...
4    {'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound...
5    {'neg': 0.091, 'neu': 0.436, 'pos': 0.473, 'co...
6    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
7    {'neg': 0.0, 'neu': 0.705, 'pos': 0.295, 'comp...
8    {'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compou...
9    {'neg': 0.0, 'neu': 0.758, 'pos': 0.242, 'comp...
Name: Review, dtype: object

In [62]:
df["polarity_score"] = df["Review"].apply(lambda x: sia.polarity_scores(x)["compound"])

In [63]:
df.head()

,Review,date,Location,Unnamed: 3,polarity_score
0,I impressed resort Great staff main resort poo...,2019-08-20,Sebastian,NaN,0.9325
1,The room nice outside need work also free brea...,2019-08-20,Los Angeles,NaN,0.8779
2,Great location I stayed hotel last three trip ...,2019-08-20,Georgia,NaN,0.6249
3,The hotel adequate stay The strip bathtub usel...,2019-08-20,NaN,NaN,0.1779
4,Great location room large spacious Parking eas...,2019-08-19,Palm Harbor,NaN,0.9260


In [64]:
df["Review"][0:10].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")

0    pos
1    pos
2    pos
3    pos
4    pos
5    pos
6    neg
7    pos
8    pos
9    pos
Name: Review, dtype: object

In [65]:
df["sentiment_label"] = df["Review"].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
df

,Review,date,Location,Unnamed: 3,polarity_score,sentiment_label
0,I impressed resort Great staff main resort poo...,2019-08-20,Sebastian,NaN,0.9325,pos
1,The room nice outside need work also free brea...,2019-08-20,Los Angeles,NaN,0.8779,pos
2,Great location I stayed hotel last three trip ...,2019-08-20,Georgia,NaN,0.6249,pos
3,The hotel adequate stay The strip bathtub usel...,2019-08-20,NaN,NaN,0.1779,pos
4,Great location room large spacious Parking eas...,2019-08-19,Palm Harbor,NaN,0.9260,pos
...,...,...,...,...,...,...
6443,staff friendly frank chef buffet bar top meeti...,2019-07-12,United States of America,NaN,0.9682,pos
6444,internet barely worked didnt work,2019-07-12,United States of America,NaN,0.0000,neg
6445,location great bed uncomfortable bed ever,2019-07-10,United States of America,NaN,0.3612,pos
6446,early check staff accommodating able check sev...,2019-07-10,United States of America,NaN,0.0000,neg


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Review           6448 non-null   object        
 1   date             6448 non-null   datetime64[ns]
 2   Location         1711 non-null   object        
 3   Unnamed: 3       0 non-null      float64       
 4   polarity_score   6448 non-null   float64       
 5   sentiment_label  6448 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 302.4+ KB


In [67]:
df["sentiment_label"].value_counts()[1]

1796

In [68]:
## % of positive tweets->the % of customers gave satisfactory reviews
(df["sentiment_label"].value_counts()[0]/(df["sentiment_label"].value_counts()[0]+df["sentiment_label"].value_counts()[1]))*100

72.14640198511167

In [69]:
## % of negative sentiment-> % of customers gave unsatisfactory reviews
(df["sentiment_label"].value_counts()[1]/(df["sentiment_label"].value_counts()[0]+df["sentiment_label"].value_counts()[1]))*100

27.853598014888338

In [70]:
sent = df['sentiment_label'].values
sent

array(['pos', 'pos', 'pos', ..., 'pos', 'neg', 'pos'], dtype=object)

In [74]:
## The mean polarity score depicts the overall sentiment of the customer reviews which comes out to be less than neutral
df['polarity_score'].mean()

0.4394559398263027

### Now segregating positive and negative feedbacks and understanding what contributes to the respective feedbacks

In [75]:
pos = df[df['sentiment_label']=='pos']
neg = df[df['sentiment_label']=='neg']

In [76]:
neg

,Review,date,Location,Unnamed: 3,polarity_score,sentiment_label
6,Old Musty Motel Bath need update asap Read Rea...,2019-08-19,NaN,NaN,0.0000,neg
21,The room tiny The building old worn inside Nee...,2019-08-18,NaN,NaN,-0.0320,neg
23,基本的に問題なしでした 清潔で防音も普通な方で水回りも清潔でシャワーの水圧も問題なしです た...,2019-08-18,NaN,NaN,0.0000,neg
29,En general excelente Fantástica situación Dos ...,2019-08-17,España,NaN,-0.2960,neg
31,Les chambres familiales sont pratiques nous ét...,2019-08-17,Montréal,NaN,-0.4588,neg
...,...,...,...,...,...,...
6436,near airport round clock shuttle airport coffe...,2019-07-14,Malaysia,NaN,0.0000,neg
6437,bed cystomer service unprofessional disrespect...,2019-07-14,United States of America,NaN,-0.7964,neg
6442,hmm proximity airport time shuttle service shu...,2019-07-13,Trinidad and Tobago,NaN,-0.5095,neg
6444,internet barely worked didnt work,2019-07-12,United States of America,NaN,0.0000,neg


In [77]:
pos['Review'] = pos['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
drops = pd.Series(' '.join(pos['Review']).split()).value_counts()[-250:]
pos['Review'] = pos['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in drops))
pos["Review"].apply(lambda x: TextBlob(x).words).head()
pos['Review'] = pos['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
tf_pos = pos["Review"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
tf_pos.columns = ["words", "tf"]
tf_pos["tf"].describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

count    9110.000000
mean       13.118332
std       114.155326
min         1.000000
5%          1.000000
10%         1.000000
25%         1.000000
50%         1.000000
75%         4.000000
80%         6.000000
90%        17.000000
95%        43.000000
99%       176.910000
max      7578.000000
Name: tf, dtype: float64

In [78]:
tf_pos.sort_values("tf", ascending = False).head(40)

,words,tf
2,Read,7578.0
9,le,3853.0
28,room,3020.0
21,The,2631.0
11,I,2515.0
29,hotel,2125.0
0,staff,1313.0
122,clean,1131.0
7,location,1100.0
41,stay,1088.0


In [79]:
neg['Review'] = neg['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
drops = pd.Series(' '.join(neg['Review']).split()).value_counts()[-250:]
neg['Review'] = neg['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in drops))
neg["Review"].apply(lambda x: TextBlob(x).words).head()
neg['Review'] = neg['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
tf_neg = neg["Review"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
tf_neg.columns = ["words", "tf"]
tf_neg["tf"].describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

count    8973.000000
mean        5.223782
std        42.257974
min         1.000000
5%          1.000000
10%         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
80%         3.000000
90%         8.000000
95%        16.000000
99%        60.280000
max      3034.000000
Name: tf, dtype: float64

In [80]:
tf_neg.sort_values("tf", ascending = False).head(40)

,words,tf
0,Read,3034.0
4,le,1556.0
16,room,1079.0
27,I,1012.0
8,The,671.0
26,hotel,606.0
72,de,275.0
438,night,228.0
150,stay,213.0
476,We,208.0


In [259]:
## Percentage of customers who were not comfortable to share their location whose feedback was positive in nature
((pos['Location'].isnull().sum())/len(df['Location']))*100

52.62096774193549

In [260]:
## Percentage of customers who were not comfortable to share their location whose feedback was negative in nature
((neg['Location'].isnull().sum())/len(df['Location']))*100

20.843672456575682

In [96]:
## Percenatage of customers not comfortable to share their location
a=((df['Location'].isna().sum())/len(df['Location']))*100
a

73.46464019851116

In [95]:
## Percentage of customers who only entered the value of Location as 'United States of America'
b = len(df[df['Location']=='United States of America'])/len(df['Location'])*100
b

1.7990074441687345

In [94]:
## Percentage of customers who were not comfortable to share their location
c=100-a-b
c

24.736352357320104

1. 72% of the feedback reveals the satisfaction of the customers while 28% of the feedbacks weren't satisfactory

2. Surprisingly, The overall feedback on average has been neutral which depicts the need for improvements

3. 73% of the customers were not comfortable to share their location in the feedback. Even if they do, less than 2% enters the location as USA. The rest of the 25% population are open to share their location.

4. The staff, location and cleanliness significantly contributes to the positive feedback of the customers while parking is one of the concern for the unsatisfactory feedback and customers face difficulty in the night time

6. Based on the hypothesis on the behavior of the customers that those having negative feedback are not comfortable to share the location which comes out to be False. The causation behind it could be probably because of either time constraint or less seriousness towards filling the customer feedback